## Imports

In [1]:
from constants import FLUXES_FOLDER, SLM_FOLDER

from data_utils import process_amp_phase_data

from plot_utils import plot_map, \
                       plot_model_history, \
                       plot_fully_connected_amp_phase_prediction, \
                       plot_conv_amp_phase_prediction

from modeling_utils import create_fully_connected_architecture_for_amplitude_and_phase_reconstruction, \
                           create_convolutional_architecture_for_amplitude_and_phase_reconstruction, \
                           compile_model, \
                           train_model

from configurations import FCDropoutL2 as ModelConfig

2023-11-30 11:43:07.660423: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-30 11:43:08.276424: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Load data

In the first place, define the data paths

In [2]:
fluxes_filename = "all_fluxes.npy"
amplitudes_filename = "complexsine_pupamp.npy"
phases_filename = "complexsine_pupphase.npy"

fluxes_path = f"{FLUXES_FOLDER}/{fluxes_filename}"
amplitudes_path = f"{SLM_FOLDER}/{amplitudes_filename}"
phases_path = f"{SLM_FOLDER}/{phases_filename}"

In [3]:
train_fluxes_array, val_fluxes_array, train_amp_phase_array, val_amp_phase_array, scalers = \
    process_amp_phase_data(
        fluxes_path,
        amplitudes_path,
        phases_path,
        n_points=10000,
        normalize_flux=True,
        normalize_amplitude=True,
        shuffle=False,
        flatten_fluxes=True,
        split=True,
        val_ratio=0.1
    )

## FULLY CONNECTED

## Define configuration

In [4]:
model_configuration = ModelConfig(train_fluxes_array,
                                  train_amp_phase_array)
print(model_configuration.get_description())


	=== FCDropoutL2 ===
	*ARCHITECTURE HYPERPARAMETERS:
		-Fully Connected
		-Input shape: (1320,)
		-Output shape: (2, 96, 96)
		-Hidden layers: [1024, 2048, 2048, 2048]
		-Regularizer: L1 (0.05)
		-Hidden Layers Activation: relu
		-Output Layer Activation: linear
		-Batch Normalization: False
		-Dropout: True 0.1
	
	*COMPILATION HYPERPARAMETERS:
		-Optimizer: ADAM lr=0.01, beta_1=0.9, beta_2=0.999
		-Loss Function: MSE
		-Metric: MSE
	
	* TRAINING HYPERPARAMETERS:
		-Epochs: 500
		-Batch size: 128
		-Callbacks:
			-ReduceLROnPlateau: MSE 30 x0.1
			-Early Stop: MSE 100
	


2023-11-30 11:43:12.513649: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-30 11:43:12.544147: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-30 11:43:12.544409: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

## Instantiate model

In [5]:
model = create_fully_connected_architecture_for_amplitude_and_phase_reconstruction(
    *model_configuration.unpack_architecture_hyperparameters()
)


## Compile model

In [6]:
compile_model(
    model,
    *model_configuration.unpack_compilation_hyperparameters()
)

## Train model

In [ ]:
history = train_model(
    model,
    train_fluxes_array,
    train_amp_phase_array,
    val_fluxes_array,
    val_amp_phase_array,
    *model_configuration.unpack_training_hyperparameters()
)

2023-11-30 11:43:13.753599: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 663552000 exceeds 10% of free system memory.
2023-11-30 11:43:14.335525: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 663552000 exceeds 10% of free system memory.


Epoch 1/500


2023-11-30 11:43:16.138257: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-11-30 11:43:16.147273: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fc0add5f250 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-30 11:43:16.147296: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2023-11-30 11:43:16.151718: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-30 11:43:16.329158: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-11-30 11:43:16.456509: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifeti

71/71 [==============================] - 4s 24ms/step - loss: 81.3529 - mean_squared_error: 74.1521 - val_loss: 9.4710 - val_mean_squared_error: 0.8601 - lr: 0.0100
Epoch 2/500
71/71 [==============================] - 1s 20ms/step - loss: 8.6589 - mean_squared_error: 0.7817 - val_loss: 7.9558 - val_mean_squared_error: 0.7720 - lr: 0.0100
Epoch 3/500
71/71 [==============================] - 1s 20ms/step - loss: 7.5432 - mean_squared_error: 0.7585 - val_loss: 7.2219 - val_mean_squared_error: 0.7753 - lr: 0.0100
Epoch 4/500
71/71 [==============================] - 1s 20ms/step - loss: 6.9246 - mean_squared_error: 0.7560 - val_loss: 6.6812 - val_mean_squared_error: 0.7753 - lr: 0.0100
Epoch 5/500
71/71 [==============================] - 1s 20ms/step - loss: 6.4475 - mean_squared_error: 0.7527 - val_loss: 6.2630 - val_mean_squared_error: 0.7771 - lr: 0.0100
Epoch 6/500
71/71 [==============================] - 1s 20ms/step - loss: 6.0858 - mean_squared_error: 0.7597 - val_loss: 6.1084 - val_

71/71 [==============================] - 1s 20ms/step - loss: 7.6640 - mean_squared_error: 0.7475 - val_loss: 7.4826 - val_mean_squared_error: 0.7691 - lr: 0.0100
Epoch 48/500
71/71 [==============================] - 1s 20ms/step - loss: 7.3210 - mean_squared_error: 0.7568 - val_loss: 7.1728 - val_mean_squared_error: 0.7691 - lr: 0.0100
Epoch 49/500
71/71 [==============================] - 1s 20ms/step - loss: 16.1349 - mean_squared_error: 9.4490 - val_loss: 167.3004 - val_mean_squared_error: 158.2292 - lr: 0.0100
Epoch 50/500
71/71 [==============================] - 1s 20ms/step - loss: 14.0493 - mean_squared_error: 4.0544 - val_loss: 10.2746 - val_mean_squared_error: 0.7755 - lr: 0.0100
Epoch 51/500
71/71 [==============================] - 1s 20ms/step - loss: 9.8073 - mean_squared_error: 0.7533 - val_loss: 9.4271 - val_mean_squared_error: 0.7706 - lr: 0.0100
Epoch 52/500
71/71 [==============================] - 1s 20ms/step - loss: 9.0978 - mean_squared_error: 0.7446 - val_loss: 8.8

## Results

In [ ]:
plot_model_history(history)

In [ ]:
n = 106
plot_fully_connected_amp_phase_prediction(
    model,
    val_fluxes_array[n],
    val_amp_phase_array[n][0],
    val_amp_phase_array[n][1]
    )

In [ ]:
n = 106
plot_fully_connected_amp_phase_prediction(
    model,
    train_fluxes_array[n],
    train_amp_phase_array[n][0],
    train_amp_phase_array[n][1]
    )